In [ ]:
# Install necessary libraries
!pip install spacy python-docx pdfplumber gradio
!python -m spacy download en_core_web_sm

# Import necessary modules
import gradio as gr
import spacy
import pdfplumber
import docx

# Load SpaCy's English NLP model
nlp = spacy.load("en_core_web_sm")

# Function to extract text from PDF resumes
def extract_text_from_pdf(pdf_file):
    with pdfplumber.open(pdf_file.name) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to extract text from DOCX resumes
def extract_text_from_docx(docx_file):
    doc = docx.Document(docx_file.name)
    text = ""
    for para in doc.paragraphs:
        text += para.text
    return text

# Function to analyze resume and extract key information
def analyze_resume(text):
    # Process the resume text
    doc = nlp(text)

    # Extract Named Entities (e.g., skills, organizations, locations, dates)
    entities = [ent.text for ent in doc.ents]

    # Extract skills and experiences using specific keywords
    skills = []
    experiences = []

    # Keywords for basic skill extraction (can be customized further)
    skill_keywords = ['Python', 'Java', 'C++', 'SQL', 'JavaScript', 'HTML', 'CSS', 'AWS', 'Docker', 'React', 'Angular']
    experience_keywords = ['experience', 'worked as', 'internship', 'project']

    # Search for skills in the resume
    for skill in skill_keywords:
        if skill.lower() in text.lower():
            skills.append(skill)

    # Search for experiences in the resume
    for experience in experience_keywords:
        if experience.lower() in text.lower():
            experiences.append(experience)

    return {
        "Entities": ', '.join(entities),
        "Skills": ', '.join(skills),
        "Experiences": ', '.join(experiences)
    }

# Gradio interface function for Resume Analyzer
def gradio_resume_analyzer(resume_file):
    if resume_file.name.endswith('.pdf'):
        resume_text = extract_text_from_pdf(resume_file)
    else:
        resume_text = extract_text_from_docx(resume_file)

    result = analyze_resume(resume_text)
    return f"Entities Extracted: {result['Entities']}\nSkills: {result['Skills']}\nExperiences: {result['Experiences']}"

# Gradio interface setup
resume_interface = gr.Interface(fn=gradio_resume_analyzer,
                                inputs=gr.File(label="Upload Resume (PDF or DOCX)"),
                                outputs=gr.Textbox(label="Extracted Information"))

# Launch the Gradio interface
resume_interface.launch(inline=True)
